<a href="https://colab.research.google.com/github/kushs5497/penny-stock-screener-openinsider/blob/main/OpenInsiderScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the proper Libraries


*   BeautifulSoup
*   DateTime
* Requests
* Pandas
* tqdm



In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import requests
import pandas as pd
from tqdm import tqdm

Method to scrape data from OpenInsider

In [ ]:
def get_data_for_month(year, month, page_no = 1):
    start_date = datetime(year, month, 1).strftime('%m/%d/%Y')
    end_date = (datetime(year, month, 1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)
    end_date = end_date.strftime('%m/%d/%Y')
    data = set()
    url = f'http://openinsider.com/screener?s=&o=&pl=&ph=&ll=&lh=&fd=-1&fdr={start_date}+-+{end_date}&td=0&tdr=&fdlyl=&fdlyh=&daysago=&xp=1&xs=1&vl=&vh=&ocl=&och=&sic1=-1&sicl=100&sich=9999&grp=0&nfl=&nfh=&nil=&nih=&nol=&noh=&v2l=&v2h=&oc2l=&oc2h=&sortcol=0&cnt=1000&page={page_no}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        rows = soup.find('table', {'class': 'tinytable'}).find('tbody').findAll('tr')
    except:
        # print(f"Finished {month}/{year}")
        return data

    for row in rows:
        cols = row.findAll('td')
        if not cols:
            continue

        insider_data = {key: cols[index].find('a').text.strip() if cols[index].find('a') else cols[index].text.strip()
                        for index, key in enumerate(['transaction_date', 'trade_date', 'ticker', 'company_name',
                                                     'owner_name', 'Title', 'transaction_type', 'last_price', 'Qty',
                                                     'shares_held', 'Owned', 'Value'])}
        data.add(tuple(insider_data.values()))
    return data


In [ ]:
data = get_data_for_month(2022,1)

for year in range(2022, 2025):
  for month in range(1, 13):
    if (year == 2024 and month>=5):
      continue
    data_size = -1
    page_num = 1
    while data_size != 0:
      data_new = get_data_for_month(year,month,page_no = page_num)
      data.update(data_new)
      data_size = len(data_new)
      page_num = page_num + 1

print(f"Retrived {len(data)} entries from OpenInsider")

column_names = ['X', 'Filing Date', 'Trade Date', 'Ticker', 'Company Name', 'Insider Name', 'Title', 'Trade Type', 'Price', 'Qty', 'Owned', 'DeltaOwn']
openInsiderDF = pd.DataFrame(data, columns = column_names)
openInsiderDF

Retrived 113165 entries from OpenInsider


,X,Filing Date,Trade Date,Ticker,Company Name,Insider Name,Title,Trade Type,Price,Qty,Owned,DeltaOwn
0,,2023-05-16 20:46:04,2023-05-12,ARES,Ares Management Corp,Bush Antoinette Cook,Dir,P - Purchase,$81.69,"+1,234","18,369",+7%
1,D,2022-11-29 16:34:51,2022-11-29,APLD,"Applied Blockchain, Inc.",Cummins Wes,"CEO; COB, 10%",P - Purchase,$2.02,"+30,000","22,562,754",0%
2,,2022-11-23 19:35:08,2022-11-21,CMPO,"Composecure, Inc.",Basile Donald G,10%,S - Sale,$4.86,"-101,633","4,291,537",-2%
3,,2023-05-09 16:17:19,2023-05-05,CNP,Centerpoint Energy Inc,Pound Ted,Dir,S - Sale,$30.50,"-2,770","46,541",-6%
4,M,2022-09-09 16:49:46,2022-09-07,OPFI,Oppfi Inc.,Schwartz Theodore G,"Dir, 10%",P - Purchase,$2.67,"+9,123","34,065,911",0%
...,...,...,...,...,...,...,...,...,...,...,...,...
113160,,2022-02-22 07:52:54,2022-02-18,NOTV,"Inotiv, Inc.",Garrett Michael,Chief Commercial Officer,P - Purchase,$23.50,"+1,000","1,000",New
113161,,2022-03-31 14:34:40,2022-03-29,CMT,Core Molding Technologies Inc,Smith Andrew O,Dir,P - Purchase,$11.11,"+2,000","67,365",+3%
113162,M,2024-02-28 21:04:42,2024-02-26,SPRY,"Ars Pharmaceuticals, Inc.",Lowenthal Richard E,"Pres, CEO, 10%",S - Sale,$9.02,"-99,478","12,949,898",-1%
113163,,2024-04-17 16:22:52,2024-04-15,COIN,"Coinbase Global, Inc.",Choi Emilie,"Pres, COO",S - Sale,$248.00,"-1,500","309,919",0%


In [ ]:
# Set 'Filing Date' to date format (i.e. remove time format)
openInsiderDF['Filing Date'] = pd.to_datetime(openInsiderDF['Filing Date'], utc=True)
openInsiderDF['Trade Date'] = pd.to_datetime(openInsiderDF['Trade Date'], utc=True)

In [ ]:
import yfinance as yf

min_date = datetime(2024,1,1)
ticker_list = openInsiderDF['Ticker'].unique().tolist()
print(f"{len(ticker_list)} Stocks starting from {min_date}")

# download_price_data = yf.download(ticker_list, start = min_date)[['Open','High','Low','Close']]
download_price_data = yf.download(ticker_list, start = min_date)
download_price_data

'''
download_price_data = download_price_data.stack(level=1).reset_index().rename(columns={'level_1':'symbol'})
download_price_data.to_csv("download_price_data.csv",index = False)
download_price_data

download_price_data['Date'] = pd.to_datetime(download_price_data['Date']).dt.date
openInsiderDF['Date'] = pd.to_datetime(openInsiderDF['Filing Date']).dt.date + timedelta(days=1)
open_insider_day_info = openInsiderDF.merge(download_price_data, how='inner', on=['Date','Ticker'])
open_insider_day_info
'''

5179 Stocks starting from 2024-01-01 00:00:00


[*******               14%%                      ]  710 of 5179 completedERROR:yfinance:Could not get exchangeTimezoneName for ticker '.' reason: 'chart'
[*********************100%%**********************]  5179 of 5179 completed
ERROR:yfinance:
705 Failed downloads:
ERROR:yfinance:['NCAC', 'RVSP', 'PEAK', 'FLME', 'JRVS', 'EFHT', 'MYOV', 'AGTC', 'IMRA', 'PZN', 'KMPH', 'BBLN', 'WSFT', 'PSXP', 'HSC', 'AKICU', 'DSEY', 'SCU', 'TBK', 'MRTX', 'RKLY', 'HVBC', 'FREQ', 'GRIL', 'VIHD', 'AFIN', 'NOAC', 'NLS', 'SPNE', 'NPTN', 'CRTX', 'TIK', 'LLNW', 'CLIN', 'MTMT', 'CLXT', 'CCHW', 'PRET', 'CCMP', 'JUPW', 'GNUS', 'ZEN', 'ID', 'PLNHF', 'ICVX', 'PSTI', 'SGLB', 'NATI', 'AGIL', 'XSPA', 'ADEX.', 'JUSH.', 'KERN', 'CSTA', 'POLY', 'NVIV', 'RFP', 'GTYH', 'EXTN', 'LVOX', 'OXAC', 'BRDS', 'RHE.A', 'HYRE', 'VCRA', 'SQZ', 'TLMD', 'NCR', 'DHHC', 'REVRQ', 'MTBC', 'KAL', 'TYME', 'GPP', 'CBTX', 'SBTX', 'MCFE', 'DISA', 'SAL', 'LFG', 'BOXD', 'ABST', 'GRAY', 'LSFP', 'ZEST', 'OEG', 'RUBY', '.', 'VAQC', 'DMS', 'ADES', 'SUM

'\ndownload_price_data = download_price_data.stack(level=1).reset_index().rename(columns={\'level_1\':\'symbol\'})\ndownload_price_data.to_csv("download_price_data.csv",index = False)\ndownload_price_data\n\ndownload_price_data[\'Date\'] = pd.to_datetime(download_price_data[\'Date\']).dt.date\nopenInsiderDF[\'Date\'] = pd.to_datetime(openInsiderDF[\'Filing Date\']).dt.date + timedelta(days=1)\nopen_insider_day_info = openInsiderDF.merge(download_price_data, how=\'inner\', on=[\'Date\',\'Ticker\'])\nopen_insider_day_info\n'

In [ ]:
import yfinance as yf

def get_shares_info(row):
    ticker = row['Ticker']
    filing_date = row['Trade Date']
    try:
        shares_info = yf.Ticker(ticker).get_shares_full(start=filing_date-timedelta(days = 1), end=filing_date)
        return shares_info
    except KeyError:
        # Handle the case where share information is not available
        print(f"Share information not available for ticker: {ticker} on date: {filing_date}")
        return None

def get_stock_data_on_date(row):
  ticker = row['Ticker']
  date = row['Trade Date']
  stock_data = yf.Ticker(ticker).history(period='1d', start=date, end=date+timedelta(days=1))
  if not stock_data.empty:
    return pd.Series([stock_data['Open'].values[0],
                        stock_data['High'].values[0],
                        stock_data['Low'].values[0],
                        stock_data['Close'].values[0]],
                        index=['Open', 'High', 'Low', 'Close'])
  else:
    return pd.Series([None, None, None, None], index=['Open', 'High', 'Low', 'Close'])

In [ ]:
# openInsiderDF['Shares Info'] = openInsiderDF.apply(get_shares_info, axis=1)
openInsiderDF[['Open', 'High', 'Low', 'Close']] = openInsiderDF.apply(get_stock_data_on_date, axis=1)
openInsiderDF

ERROR:yfinance:MAXR: No timezone found, symbol may be delisted
ERROR:yfinance:FOMC: No timezone found, symbol may be delisted
ERROR:yfinance:FOMC: No timezone found, symbol may be delisted
ERROR:yfinance:BTRS: No timezone found, symbol may be delisted
ERROR:yfinance:SPPI: No timezone found, symbol may be delisted
ERROR:yfinance:NEWR: No timezone found, symbol may be delisted
ERROR:yfinance:CINC: No timezone found, symbol may be delisted
ERROR:yfinance:PATI: No timezone found, symbol may be delisted
ERROR:yfinance:FREQ: No timezone found, symbol may be delisted
ERROR:yfinance:LHDX: No timezone found, symbol may be delisted
ERROR:yfinance:AVLR: No timezone found, symbol may be delisted
ERROR:yfinance:VACC: No timezone found, symbol may be delisted
ERROR:yfinance:BLI: No timezone found, symbol may be delisted
ERROR:yfinance:SUMO: No timezone found, symbol may be delisted
ERROR:yfinance:CGRN: No timezone found, symbol may be delisted
ERROR:yfinance:IIVI: No timezone found, symbol may be de

KeyboardInterrupt: 